In [ ]:
!pip install google-cloud-bigquery

# Configurações iniciais

In [ ]:
from google.colab import auth
auth.authenticate_user()

from google.cloud import bigquery
import os
job_config = bigquery.QueryJobConfig(
    # Run at batch priority, which won't count toward concurrent rate limit.
    priority=bigquery.QueryPriority.BATCH
)


# Agora você pode criar o cliente BigQuery
client = bigquery.Client()

In [ ]:
project_id = "projeto-sql-431918"
client = bigquery.Client(project=project_id)

# Carregar lista de CNPJs do arquivo exportado

In [ ]:
import pandas as pd
cnpjs_df = pd.read_excel('New Report (4).xlsx')  # Ou pd.read_excel para arquivos Excel
cnpjs_df['Cadastro federal do cliente'] = cnpjs_df['Cadastro federal do cliente'].fillna(0).astype(int)
cnpjs_list = cnpjs_df['Cadastro federal do cliente'].tolist()
## print(cnpjs_df.head())
print(cnpjs_list)
print(cnpjs_df.dtypes['Cadastro federal do cliente'])

[2558157000162, 60613478000119, 26325143000167, 7358761000169, 20412770000159, 30306294000145, 10844817000108, 13467877000139, 20588260000137, 10140658000152, 5265763000188, 73178600000118, 11471792000107, 57756694000109, 67003673000176, 47283189000130, 26610150000100, 609783000105, 67647180000179, 15176391000177, 48109110000112, 1795309000188, 19370970000107, 4052108000189, 7465490000140, 9129455000186, 8065557000112, 27275126000125, 8801621000186, 9531894000110, 46358602000116, 19010359000178, 2520543000165, 7731758000149, 8842690000138, 12461756000117, 56045198000184, 13486793000142, 89637490000145, 11758741000152, 24814369000104, 11472927000140, 19364086000160, 21599995000129, 3594123000196, 45246410000155, 18259351000187, 28921582000168, 43854116006484, 58890252000113, 1650515000280, 8870769000172, 7728655000120, 33400689000109, 22610500000188, 6975199000150, 11414555000104, 10302599000171, 7253654000176, 7939675000140, 534198000606, 7368550000107, 53586822000162, 19276023000151, 

# Converter lista de CNPJs em string formatada para a consulta SQL

In [ ]:
cnpjs_str = ",".join([f"'{cnpj}'" for cnpj in cnpjs_list])

# Escrever a consulta SQL

In [ ]:
query = f"""
SELECT *
FROM `projeto-sql-431918.basedosdados_cnpj_estabelecimentos.cnpj`
WHERE cnpj IN ({cnpjs_str})
"""

# Executar a consulta no BigQuery

In [ ]:
query_job = client.query(query)

results = query_job.result()

# Exibir os resultados
for row in results:
    print(row)

Forbidden: 403 Quota exceeded: Your project exceeded quota for free query bytes scanned. For more information, see https://cloud.google.com/bigquery/docs/troubleshoot-quotas; reason: quotaExceeded, location: unbilled.analysis, message: Quota exceeded: Your project exceeded quota for free query bytes scanned. For more information, see https://cloud.google.com/bigquery/docs/troubleshoot-quotas

Location: US
Job ID: 445804a4-bc28-45f9-8b78-82e257205da0


In [ ]:
## results = query_job.result()

## for row in results:
    ## print(row)

In [ ]:
results = query_job.result().to_dataframe()

# Salvar os resultados em um arquivo ou processar conforme necessário

In [ ]:
results.to_csv('resultados.csv', index=False)